In [ ]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 150

In [ ]:
keep_cols = ['node_id', 'sourceID', 'name', 'countries', 'country_codes']
dtypes = = {'node_id': 'int32', 'sourceID':'category', 'name':'object',
            'countries':'category', 'country_codes':'category'}

bahamas_intermediary_raw = pd.read_csv('../data/raw/bahamas_leaks/bahamas_leaks.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
offshore_intermediary_raw = pd.read_csv('../data/raw/offshore_leaks/offshore_leaks.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
panama_intermediary_raw = pd.read_csv('../data/raw/panama_papers/panama_papers.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)
paradise_intermediary_raw = pd.read_csv('../data/raw/paradise_papers/paradise_papers.nodes.intermediary.csv', 
                                 usecols = keep_cols,
                                 dtype=dtypes)

### To Do

- Standardize the names (Title case)
- There are companies and people in the name field. Do you want to break this into different tables? One with specific people and one with companies?
- Break peope into first and last name?
- Companies are broken out by their branch office. Do you want to make a Corporate column that unites these offices?